In [118]:
import xml.etree.cElementTree as ET
import re
from collections import defaultdict
import cerberus
import schema
import csv
import codecs
import sqlite3
import pprint

We start by importing the file we'll be using for our wrangling. I chose the city of L.A as it was a city I grew attached to during my Masters Program.

In [67]:
OSM_PATH = "C:\Users\shawar\Documents\la_map.osm"

I created a function below to show us the different types of tag attributes in the way and node tags and their count and present them as a dictionary. The function also allows us to check the different values for each tag key. 

In [68]:
def number_types(tag):
    num_dict = defaultdict(int)
    types_dict = defaultdict(set)
    for event, elem in ET.iterparse(OSM_PATH, events = ('start',)):
        if elem.tag == 'way':
            for tags in elem.iter(tag):
                num_dict[tags.get('k')] += 1
                types_dict[tags.get('k')].add(tags.get('v'))
    return num_dict, types_dict

In [114]:
num_types, types_values = number_types('tag')
count = 0
for key, value in num_types.items():
    print key, value 
    if count == 25:
        break    
    count += 1                                          #small sample of different keys and their count

caltrans:type 1
tiger:source 2040
maxspeed 627
source_ref:oneway 3
housing_type 1
source:private 2
turn:lanes:forward 322
placement:forward 2
is_in 23
building:height 1
maxspeed:truck 2
tiger:AWATER 4
created_by 235
hov:minimum 4
name:full 2
attribution 34
tiger:county 4761
name:uk 1
motor_vehicle 33
faa 1
railway:track_ref 3
source_ref:bicycle 14
addr:street 394
source:name 17
level 15
cutline 5


What I first noticed was odd in the data was that there were many tags with the name 'FIXME'.

We also see the many ways in which the same kind of data has different ways of describing them. For example, to describe the county in which the particular location is, we have - ('addr:county', 'is_in:county', 'gnis:county_name')

I decided to look at important keys in the above dictionary a little more closely. (name, addr types, amenities, zipcodes, etc.)

In [171]:
count = 0
for event, elem in ET.iterparse(OSM_PATH, events= ('start',)):
    if count == 6:
        break
    else:
        if elem.tag == 'node' or elem.tag == 'way':
            for tag in elem.iter('tag'):
                if tag.get('k') == 'fixme':
                    count += 1
                    print tag.attrib

{'k': 'fixme', 'v': 'split this intersection up'}
{'k': 'fixme', 'v': "what's here now? Lot is bulldozed on Bing"}
{'k': 'fixme', 'v': 'Location'}
{'k': 'fixme', 'v': 'continue'}
{'k': 'fixme', 'v': 'add height, ref pole number'}
{'k': 'fixme', 'v': 'is this the right location?'}


In [122]:
count = 0
for key, value in types_values.items():                 #sample of keys and their different values
    pprint.pprint(key), pprint.pprint(value) 
    if count == 20:
        break    
    count += 1  

'caltrans:type'
set(['HMS,LSMS'])
'tiger:source'
set(['TIGER 2011 ogr2osm import',
     'tiger_import_dch_v0.6_20070809',
     'tiger_import_jan_v0.5_20120613'])
'maxspeed'
set(['10 mph',
     '112',
     '127',
     '14 mph',
     '15 mph',
     '20',
     '20 mph',
     '25',
     '25 mph',
     '30 mph',
     '35 mph',
     '37',
     '38',
     '40',
     '40 mph',
     '45',
     '45 mph',
     '48',
     '5 mph',
     '50',
     '50 mph',
     '55 mph',
     '56',
     '60 mph',
     '65 mph',
     '70 mph',
     '96.5'])
'source_ref:oneway'
set(['AM909_DSCU2397', 'AM909_DSCU4562', 'AM909_DSCU4766'])
'housing_type'
set(['condominium'])
'source:private'
set(['LA County LA County Street Centerline & Address File (http://egis3.lacounty.gov/dataportal/2011/12/09/2011-la-county-street-centerline-street-address-file/)',
     'survey;image;LACA;LACDPW'])
'turn:lanes:forward'
set(['left;through|',
     'left;through|right|right',
     'left;through||',
     'left|',
     'left|left;throu

We see some of the tags for an element have address streets written as a whole as well as split up. I noticed that the source for most of this data was the Tiger GPS system.

Some tags have addresses which give a direction as abrreviations such as N, S, E, W whereas other places have North, South, East, West.

Some element tags have different notations for name keys. Some have the key as 'name' whereas others have 'name_1'.

Phone numbers for different elements have different formats too. ('+1 xxx-xxx-xxxx', '1 xxx-xxx-xxxx', 'xxx xxx xxxx')

Some elements have zipcode tags in their child tags, some have postcode tags. There are also zipcode_left and zipcode_right child tags for the same element 

One of the tags has an entire note ( under the key 'note') referring to inconsistencies.

Mainly, and most importantly, street addresses have different forms too. Some have abbreviated forms like 'St', 'Rd' whereas others have 'Street', 'Road'.

I will look into some of these tags in more depth and try to fix most of them.

I wanted to investigate the street addresses issue for the dataset. Looking through the data I noticed that street names were present in 'name', 'name_1' and 'addr:street' tags for way elements. For the node elements, street names are found in the 'addr:street' k values of the tag child elements. 

I decided to work with the values associated with the 'addr:street' keys.

In [133]:
def is_street(element):    #this will check if a key represents a street
    return element.get('k') == 'addr:street'

reg_exp = r'(?i)(\b\S+|\d+(\.?))$'   #This regular expressions will take a street name at return the last word.
                                     #This is done to check with the common_street_types variable given below.

In [136]:
common_street_types = ['Avenue', 'Boulevard', 'Court', 'Drive', 'Lane',
                       'Parkway','Place', 'Road', 'Square', 'Street', 'Trail', 'Way']

In [95]:
def get_street(reg_ex, street_name):
    match = re.search(reg_ex, street_name)
    if match:
        if match.group() not in common_street_types:
            if match.group() not in street_corrections.keys():
                return street_name
            else:
                return re.sub(reg_ex, street_corrections[match.group()], street_name)
        else:
            return street_name

The function above takes a street name, checks if the last word of the street is a common street type. If not, it checks whether the last word is an abbreviated form as given in the street_corrections dictionary below. If it is, it corrects it accordingly and if not, just returns the street name

In [146]:
count = 0    #sample of correcting street names
for event, elem in ET.iterparse(OSM_PATH, events = ('start',)):
    if count == 20:
        break
    else:
        if elem.tag =='tag' or elem.tag == 'node':
            for tag in elem.iter('tag'):
                if is_street(tag):
                    count += 1
                    print tag.attrib['v'], '---------->',  get_street(reg_exp, tag.get('v'))

N Varney St ----------> N Varney Street
N Varney St ----------> N Varney Street
Glenwood Road ----------> Glenwood Road
Glenwood Road ----------> Glenwood Road
Briercrest Avenue ----------> Briercrest Avenue
Briercrest Avenue ----------> Briercrest Avenue
Monte Vista Street ----------> Monte Vista Street
Monte Vista Street ----------> Monte Vista Street
Venice Boulevard ----------> Venice Boulevard
Venice Boulevard ----------> Venice Boulevard
Santa Anita Avenue ----------> Santa Anita Avenue
Santa Anita Avenue ----------> Santa Anita Avenue
Carlos Ave ----------> Carlos Avenue
Carlos Ave ----------> Carlos Avenue
Greystone Dr ----------> Greystone Drive
Greystone Dr ----------> Greystone Drive
San Gorgonio Dr ----------> San Gorgonio Drive
San Gorgonio Dr ----------> San Gorgonio Drive
Brookside Ave ----------> Brookside Avenue
Brookside Ave ----------> Brookside Avenue


In [137]:
street_corrections = {'St': 'Street',  'Rd': 'Road',
                      'Wy' :'Way', 'Pkwy': 'Parkway', 'Pl': 'Place',
                      'Ln': 'Lane', 'Dr': 'Drive', 'Ct' : 'Court',
                      'Blvd.' : 'Boulevard', 'Blvd':'Boulevard', 'Blv.' : 'Boulevard', 'Ave':'Avenue',}

While skimming through the zipcodes, I noticed a bunch of values with more than just the standard 5 digits. So i decided to take these values and just return the first part of the zipcode which matched the conventional 5 digit format

In [97]:
def is_zip(elem):  #function that uses a regualr expression to check for zip/post code patterns in the key name
    if re.search(r'(?i)(zip|:post)', elem.get('k')):
        return elem.get('k')

In [154]:
def get_zip(elem):  #This function taks a zipcode and returns the first 5 digits of the zipcode
    reg_ex_zip = r'(?P<zip>\d{5})((\S(\s)?)(\d+))*'      #regular expression to return the first 5 digits
    return re.sub(reg_ex_zip, r'\g<zip>', elem.get('v'))

In [153]:
count = 0    #sample of how the zipcode cleaning returns values
for event, elem in ET.iterparse(OSM_PATH, events = ('start',)):
    if count == 10:
        break
    else:
        if elem.tag =='node' or elem.tag == 'way':
            for tag in elem.iter('tag'):
                if is_zip(tag) and ':' in tag.get('v'):
                    count += 1
                    print tag.get('v'), '--------->', get_zip(tag)

90022:90640 ---------> 90022
91730:91737 ---------> 91730
90602:90605 ---------> 90602
90602:90605 ---------> 90602
92625:92651 ---------> 92625
92625:92651 ---------> 92625
90274:90717 ---------> 90274
90274:90717 ---------> 90274
90712:90807 ---------> 90712
90712:90807 ---------> 90712


When I looked at various phone numbers , I noticed that they had different formats and they are also under different key names. I decided to collect all the phone numbers and return them in a more standard +1 xxx-xxx-xxxx format. The function below does just that with the help of the regular expressions phone_re

In [99]:
def get_phone(elem):  #function to look through phone numbers
    if elem.get('k') == 'phone' or elem.get('k') == 'fax':
        return elem.get('v')      

In [161]:
### This regular expression finds all the values that resemble the pattern of a phone number with the area code in the U.S
phone_re = r'(?P<area>\d{3})(.+)?(?P<val2>\d{3})(.+)?(?P<val3>\d{4})'   

In [100]:
def correct_phone(elem):  ### This function takes in a phone number of a particular format and converts it to the standard format
    numbers = get_phone(elem)
    match = re.search(phone_re, numbers)
    if match:
            correct_numbers = re.sub(phone_re, r'+1 \g<area>-\g<val2>-\g<val3>', match.group())
            return correct_numbers

In [162]:
count = 0   #sample of phone number corrections
for event, elem in ET.iterparse(OSM_PATH, events = ('start',)):
    if count == 20:
        break
    else:
        if elem.tag == 'node' or elem.tag == 'way':
            for tag in elem.iter('tag'):
                if get_phone(tag):
                    count += 1
                    print tag.get('v'), '--------->', correct_phone(tag)

+1 818 244 2113 ---------> +1 818-244-2113
323-255-5416 ---------> +1 323-255-5416
310 390-3454 ---------> +1 310-390-3454
1-625- 574-1613 ---------> +1 625-574-1613
+1 909 390 6788 ---------> +1 909-390-6788
+1-805-684-9909 ---------> +1 805-684-9909
+1-805-643-6615 ---------> +1 805-643-6615
323-466-3251 ---------> +1 323-466-3251
562-860-8454 ---------> +1 562-860-8454
714-990-1400 ---------> +1 714-990-1400
(714) 538-3764 ---------> +1 714-538-3764
(213) 622-3333 ---------> +1 213-622-3333
800-932-6278 ---------> +1 800-932-6278
818-338-8888 ---------> +1 818-338-8888
3104822035 ---------> +1 310-482-2035
+1-213-403-3100 ---------> +1 213-403-3100
+1-213-403-3000 ---------> +1 213-403-3000
626-446-7238 ---------> +1 626-446-7238
626-446-8222 ---------> +1 626-446-8222
+1-310-656-8500 ---------> +1 310-656-8500


This conversion makes the data look consistent and very clean

Next, I decided to look at street names that had abbreviations for their directions like E,W,N,S

In [102]:
# To fix abbreviations like N,S,E,W
reg_e_dir = r'(?i)\b[ewns]\.?\b'

In [103]:
direction_dict = {'N' : 'North', 'E.': 'East', 'S': 'South', 'E': 'East'}

In [104]:
def get_direction(regex, street_name):  #funtion that takes street names, looks for E, W, N, S and returns the full form.
    match = re.search(regex, street_name)
    if match:
        if match.group() in direction_dict.keys():
            return re.sub(regex, direction_dict[match.group()], street_name)
        else:
            return street_name
    else:
        return street_name

After looking at all the cleaning that was possible, it was time to collect the data from the osm file and put it into a csv format for further analysis 

In [1]:
before_colon = r'(?=(\:)?)\w+'  ### regular expression to find everything before the first colon 
after_colon = r'(?<=(\:)).+'   ### regular expression to find everything after the first colon

In [54]:
NODES_PATH = "C:\Users\shawar\Documents\\nodes.csv"
NODE_TAGS_PATH = "C:\Users\shawar\Documents\\nodes_tags.csv"
WAYS_PATH = "C:\Users\shawar\Documents\\ways.csv"
WAY_NODES_PATH = "C:\Users\shawar\Documents\\ways_nodes.csv"
WAY_TAGS_PATH = "C:\Users\shawar\Documents\\ways_tags.csv"

NODE_FIELDS = ['id', 'lat', 'lon', 'user', 'uid', 'version', 'changeset', 'timestamp']
NODE_TAGS_FIELDS = ['id', 'key', 'value', 'type']
WAY_FIELDS = ['id', 'user', 'uid', 'version', 'changeset', 'timestamp']
WAY_TAGS_FIELDS = ['id', 'key', 'value', 'type']
WAY_NODES_FIELDS = ['id', 'node_id', 'position']
problemchars = re.compile(r'[=\+/&<>;\'" \?%#$@\,\.\t\r\n]') ### regular expression to find non alphanumeric characters
SCHEMA = schema.schema

def shape_element(element, node_attr_fields=NODE_FIELDS, way_attr_fields=WAY_FIELDS):
    """Clean and shape node or way XML element to Python dict"""

    node_attribs = {}
    way_attribs = {}
    way_nodes = []
    tags = []

    

    if element.tag == 'node':
        for node in NODE_FIELDS:
            node_attribs[node] = element.attrib[node]  ### creating key-value pairs in the node_attribs dictionary 
                                                       ### where the keys are the elements in the NODE_FIELDS list 
                                                       ### and values are respective attributes
            
    
        
        for tag in element.iter('tag'):    ###iterating over each tag named 'tag'
            diction = {}
            if not re.search(problemchars, tag.get('k')):    ### if characters in the problemchars regular expression are not
                                                             ### in the tag keys, assign it to 'id' in diction
                diction['id'] = element.get('id') 
                ### keys of tags have to be treated differently based on whether there is a ':' or not. The following if 
                ### statement is to deal with that.
                
                if ':' in tag.get('k'):                      
                    if is_zip(tag):                          ### calling this function to check if it is a zipcode
                        diction['value'] = get_zip(tag)      ### calling this function to extract the zipcode value and assign
                                                             ### it to the 'value' key in diction
                            
                        ### the following two steps extracts everything before and after the first colon in the key tag 
                        ###and assigns it to the 'type' key and 'key' key of diction
                        
                        diction['type'] = re.search(before_colon, tag.get('k')).group()
                        diction['key'] = re.search(after_colon,tag.get('k')).group()
                    
                    elif is_street(tag):   ### to check if the value tag contains a street name
                        right_street = get_street(reg_exp, tag.get('v'))  ### calls function to change the street name to the 
                                                                          ### corrected format using regular expression 
                                                                          ### explained above
                            
                        diction['value'] = get_direction(reg_e_dir, right_street) ### Changing directional abbreviations 
                                                                                  ### to their full form using regular 
                                                                                  ### expression explained above
                            
                        diction['key'] = re.search(before_colon, tag.get('k')).group()
                        diction['type'] = re.search(after_colon, tag.get('k')).group()
                    else:
                        diction['key'] = tag.get('k')
                        diction['value'] = tag.get('v')
                        diction['type'] = re.search(before_colon, tag.get('k')).group()
                        
                else:  ###if no ':' in the key
                    
                    if get_phone(tag):  ###function to check if tag resembles a phone number
                        diction['value'] = correct_phone(tag) ### function that returns the phone number in the 
                                                              ### correct phone number
                            
                        diction['type'] = 'regular' ### if no ':', the rule was to return 'regular' as the type
                        diction['key'] = tag.get('k')
                    else:
                        diction['value'] = tag.get('v')
                        diction['type'] = 'regular'
                        diction['key'] = tag.get('k')
                        
                tags.append(diction)   ###appending the dictionary to 'tags' list
        return {'node': node_attribs, 'node_tags': tags}            
     
    if element.tag == 'way':
        for way in WAY_FIELDS:
            way_attribs[way] = element.attrib[way]  ### similar process to nodes above
            
            
        
        count = 0
        for tag in element.iter('nd'):
            way_nd = {}
            way_nd['id'] = element.get('id')
            way_nd['node_id'] = tag.get('ref')
            way_nd['position'] = count  ### each node of a particular way is given an ordered number from 
                                        ### 0 - n number of nodes
            way_nodes.append(way_nd)
            count += 1
        
        
        for tag in element.iter('tag'): ### looping through the 'tag' tags of each way element in the same process as the nodes
            diction = {}
            if not re.search(problemchars, tag.get('k')):
                diction['id'] = element.get('id')
                if ':' in tag.get('k'):
                    if is_zip(tag):
                        diction['value'] = get_zip(tag)
                        diction['type'] = re.search(before_colon, tag.get('k')).group()
                        diction['key']  = re.search(after_colon, tag.get('k')).group()
                    elif is_street(tag):
                        right_street = get_street(reg_exp, tag.get('v'))
                        diction['value'] = get_direction(reg_e_dir, right_street)
                        diction['key'] = re.search(before_colon, tag.get('k')).group()
                        diction['type'] = re.search(after_colon, tag.get('k')).group()
                    else:
                        diction['key'] = tag.get('k')
                        diction['value'] = tag.get('v')
                        diction['type'] = re.search(before_colon, tag.get('k')).group()
                else:
                    if get_phone(tag):
                        diction['value'] = correct_phone(tag)
                        diction['type'] = 'regular'
                        diction['key'] = tag.get('k')
                    else:
                        diction['value'] = tag.get('v')
                        diction['type'] = 'regular'
                        diction['key'] = tag.get('k')
                tags.append(diction)
        return {'way': way_attribs, 'way_nodes': way_nodes, 'way_tags': tags}
    
def get_element(osm_file, tags=('node', 'way', 'relation')):
    """Yield element if it is the right type of tag"""

    context = ET.iterparse(osm_file, events=('start', 'end'))
    _, root = next(context)
    for event, elem in context:
        if event == 'end' and elem.tag in tags:
            yield elem
            root.clear()


def validate_element(element, validator, schema=SCHEMA):
    """Raise ValidationError if element does not match schema"""
    if validator.validate(element, schema) is not True:
        field, errors = next(validator.errors.iteritems())
        message_string = "\nElement of type '{0}' has the following errors:\n{1}"
        error_strings = (
            "{0}: {1}".format(k, v if isinstance(v, str) else ", ".join(v))
            for k, v in errors.iteritems()
        )
        raise cerberus.ValidationError(
            message_string.format(field, "\n".join(error_strings))
        )


class UnicodeDictWriter(csv.DictWriter, object):
    """Extend csv.DictWriter to handle Unicode input"""

    def writerow(self, row):
        super(UnicodeDictWriter, self).writerow({
            k: (v.encode('utf-8') if isinstance(v, unicode) else v) for k, v in row.iteritems()
        })

    def writerows(self, rows):
        for row in rows:
            self.writerow(row)


# ================================================== #
#               Main Function                        #
# ================================================== #
def process_map(file_in, validate):
    """Iteratively process each XML element and write to csv(s)"""

    with codecs.open(NODES_PATH, 'w') as nodes_file, \
         codecs.open(NODE_TAGS_PATH, 'w') as nodes_tags_file, \
         codecs.open(WAYS_PATH, 'w') as ways_file, \
         codecs.open(WAY_NODES_PATH, 'w') as way_nodes_file, \
         codecs.open(WAY_TAGS_PATH, 'w') as way_tags_file:

        nodes_writer = UnicodeDictWriter(nodes_file, NODE_FIELDS)
        node_tags_writer = UnicodeDictWriter(nodes_tags_file, NODE_TAGS_FIELDS)
        ways_writer = UnicodeDictWriter(ways_file, WAY_FIELDS)
        way_nodes_writer = UnicodeDictWriter(way_nodes_file, WAY_NODES_FIELDS)
        way_tags_writer = UnicodeDictWriter(way_tags_file, WAY_TAGS_FIELDS)

        nodes_writer.writeheader()
        node_tags_writer.writeheader()
        ways_writer.writeheader()
        way_nodes_writer.writeheader()
        way_tags_writer.writeheader()

        validator = cerberus.Validator()

        for element in get_element(file_in, tags=('node', 'way')):
            el = shape_element(element)
            if el:
                if validate is True:
                    validate_element(el, validator)
                if element.tag == 'node':
                    nodes_writer.writerow(el['node'])
                    node_tags_writer.writerows(el['node_tags'])
                elif element.tag == 'way':
                    ways_writer.writerow(el['way'])
                    way_nodes_writer.writerows(el['way_nodes'])
                    way_tags_writer.writerows(el['way_tags'])


if __name__ == '__main__':
    process_map(OSM_PATH, validate=False)
    print "Done"


Done


File Sizes:

.osm File --> 158 MB                                                                                                     
.db File --> 238 MB                                                                                                        
nodes.csv --> 64.MB                                                                                                         
nodes_tags.csv --> 739 KB                                                                                                    
ways.csv --> 4.58 MB                                                                                                         
ways_nodes.csv --> 19.2 MB                                                                                                  
ways_tags.csv --> 16.4MB                                                                                           

In [90]:
# Importing everything to sqlite3

#Total number of nodes

    sqlite> SELECT count(*) FROM nodes;                 
714852

#Total number of ways

    sqlite> SELECT count(*) FROM ways;
69756
    

#Number of Unique IDs

    sqlite> SELECT count(DISTINCT(uid)) FROM(SELECT uid FROM nodes UNION ALL SELECT uid FROM ways);
1734

#Top 5 Contributors

    sqlite> SELECT user, count(user) as Contributions                                                                           
   ...> FROM (SELECT user FROM nodes UNION ALL SELECT user from ways)                                                         
   ...> GROUP BY user                                                                                                       
   ...> ORDER BY Contributions DESC                                                                                         
   ...> LIMIT 5;                                                                                                              
   
manings_labuildings|55706                                                                                                
calfarome_labuilding|53499                                                                                                      
Luis36995_labuildings|49100                                                                                                     
dannykath_labuildings|43463                                                                                                  
schleuss_imports|43115                                                                                                        

#calculating average contributions from users

      sqlite> SELECT avg(num) FROM (SELECT COUNT(*) as num
  ...> FROM (SELECT user FROM nodes UNION ALL SELECT user FROM ways) nw                                                         
  ...> GROUP BY nw.user);                                                                                                     
452.223631123919



Now let us compare the average contributions of the top 100 contributors vs the rest

TOP 100 Average                                                                                                           
      sqlite> SELECT avg(num) FROM (SELECT COUNT(*) as num
   ...> FROM (SELECT user FROM nodes UNION ALL SELECT user FROM ways) nw                                                      
   ...> GROUP BY nw.user ORDER BY num DESC LIMIT 100);                                                                        
7412.65

100- 1734 Average                                                                                                        
      sqlite> SELECT avg(num) FROM (SELECT COUNT(*) as num                                                                     
       ...> FROM (SELECT user FROM nodes UNION ALL SELECT user FROM ways) nw                                                     
       ...> GROUP BY nw.user ORDER BY num DESC LIMIT 100,1634);                                                                                                                                 
26.5250917992656


As we can see, the top contributors contribute much more than the rest. Infact, if you go a little deeper, and see the bottom 1000 people:-

    sqlite> SELECT sum(num) FROM (SELECT COUNT(*) as num                                                                       
   ...> FROM (SELECT user FROM nodes UNION ALL SELECT user FROM ways) nm                                                      
   ...> GROUP BY nm.user ORDER BY num DESC LIMIT 734,1000);
   
1970

The bottom 1000 people contributions are not even remotely close to the top contributor

#No of Subway Restaurants

    sqlite> SELECT count(*) FROM nodes_tags                                                                            
    WHERE value ='Subway';
4


#Top 10 Number of amenities

    sqlite> SELECT value, count(value) as Total FROM
   ...> (SELECT value FROM nodes_tags where key = 'amenity'                                                                     
   ...> UNION ALL SELECT value FROM ways_tags where key ='amenity')                                                             
   ...> GROUP BY value ORDER BY Total DESC                                                                                         
   ...> LIMIT 10;                                                                                                        
   
parking|247                                                                                                                     
school|95                                                                                                                       
place_of_worship|88                                                                                                             
restaurant|62                                                                                                                  
fast_food|47                                                                                                                 
fuel|25                                                                                                                        
toilets|25                                                                                                                     
cafe|23                                                                                                                         
bench|19                                                                                                                        
fire_station|15                                                                                                                

#Number of Cycle Lanes

    sqlite> SELECT value, count(*) FROM ways_tags
   ...> WHERE value ='cycleway';  
   
cycleway|34

Idea for improving OSM:

  An issue I came across while wrangling this data were the inconsistencies to explain the same type of data. One of the reasons for this is that the data is coming from different sources, for example 'tiger GPS system'. During the analysis, I noticed that zipcodes were given in a 'zipcode-right' and 'zipcode-left' format for the same way tag. Both these values were always equal. A great way to counter this and make sure that the data coming from different sources is always in a consistent format is to allow edits to made in a structured input format (Name, Street, Type, zipcode).
  
  Another curious thing I noticed was that some of the way tags for an element had the complete street address whereas in some elements, there were separate tags dividing the address. 
  
  Finally, most contributions are from a handful of users where even the bottom 1000 were far short in terms of contributions when compared to the top contributor. The top 5 contributors had contributions way above 40000, but the average contributions for the top 100 was only around 7500. The average contributions drop even further for the contributors out of the top 100 (26). Thus, we see that, where people at the top contribute in the tens of thousands, people below are hardly contributing 50. The total number of contributions of the bottom thousand is 1970, which when compared with the highest contributor (55706), shows us how much of a gap there really is in terms of contributions.
  
  Contributions need to come from more users in order to improve the data quality. One way of doing this would be to create incentives for people with low number of contributions to contribute. Another way would be to create a competitive edge by implementing a scoring system among users.

REFERENCES:

1) Openstreetmap link - https://mapzen.com/data/metro-extracts/metro/los-angeles_california/                                
2) udacity.com                                                                                                          
3) stackoverflow.com                                                                                                         
4) https://wiki.openstreetmap.org/                                                                                